<a href="https://colab.research.google.com/github/dentadelta/123/blob/master/T5_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I borrows  a few lines of codes from the below Google Colabs:

https://colab.research.google.com/github/patil-suraj/exploring-T5/blob/master/T5_on_TPU.ipynb#scrollTo=coOmS2s_xDBy

In [0]:
!pip install nlp transformers

     |████████████████████████████████| 860kB 2.8MB/s 
     |████████████████████████████████| 675kB 16.5MB/s 
     |████████████████████████████████| 63.8MB 48kB/s 
     |████████████████████████████████| 1.1MB 41.7MB/s 
     |████████████████████████████████| 890kB 33.9MB/s 
     |████████████████████████████████| 3.8MB 35.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=a30044918b5b7b9bd5c6db92b9a286591653dfdcbbec2df4fd934a3e15a851d0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


Make sure you reload the runtime after you run the above codes

Also make sure you change your runtime to GPU

In [0]:
import pandas as pd
import os
import torch
from transformers import (T5Config, T5Tokenizer, T5ForConditionalGeneration, TextDataset, DataCollator, Trainer, TrainingArguments)
import ipywidgets as widgets
import random
from typing import Dict, List, Optional, Union
import nlp
from dataclasses import dataclass
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
import re

In [0]:
import pyarrow as pa

In [0]:
pretrained_model_name = 't5-base'   # options are: t5-small, t5-base, t5-large, t5-3B, t5-11B
training_file_path = ''   # I downloaded the csv from dropbox using the below url
working_folder = '/content/'

os.environ['First_Time_Training_The_Model'] = 'True'

In [0]:
url =  'https://www.dropbox.com/s/6w5z4qvt8vytngm/training_data.csv?dl=1' # If you save your data in dropbox then anyone who has the link can access to the data in real time :P
df=pd.read_csv(url)
df = df[:10000]   # Only load the first 10000 dataset (You can train the entire dataset if you like)

# df['prefix'] = 'seq2seq'  Uncomment this line if you are not training a question to answer model

In [0]:
df

,prefix,input_text,target_text
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary
...,...,...,...
9995,Institutes of technology with different origin...,What is the name of Thammasat University's eng...,Sirindhorn International Institute of Technology
9996,Institutes of technology with different origin...,What is Thailand's only government-established...,Suranaree University of Technology
9997,Institutes of technology with different origin...,What year was Suranaree University of Technolo...,1989
9998,Institutes of technology with different origin...,What is the name of the best-known private ins...,Mahanakorn University of Technology


In [0]:
df['prefix'][0]

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [0]:
length_of_prefix = len(df['prefix'][0])
length_of_prefix  # Prefix is the context document

695

In [0]:
def add_eos_to_examples(example):
    example['input_text'] = '<{}>: <{}> </s>'.format(example['prefix'] , example['input_text'] )
    example['target_text'] = '"<{}> </s>"'.format(example['target_text'])
    return example

def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input_text'], pad_to_max_length=True, max_length=1700)     ########## Specify the maximum input lengths (context + question)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target_text'], pad_to_max_length=True, max_length=16)     ########## Specify the maximum output length
    encodings = {
        'input_ids': input_encodings['input_ids'], 
        'attention_mask': input_encodings['attention_mask'],
        'target_ids': target_encodings['input_ids'],
        'target_attention_mask': target_encodings['attention_mask']
    }
    return encodings

In [0]:
tokenizer = T5Tokenizer.from_pretrained(pretrained_model_name)

In [0]:
fields = [
          ('input_text', pa.string()),
          ('target_text', pa.string()),
          ('prefix', pa.string())
      ]

train_dataset = nlp.arrow_dataset.Dataset(pa.Table.from_pandas(df,pa.schema(fields)))
train_dataset = train_dataset.map(add_eos_to_examples)
train_dataset = train_dataset.map(convert_to_features, batched=True)
columns = ['input_ids', 'target_ids', 'attention_mask', 'target_attention_mask']
train_dataset.set_format(type='torch', columns=columns)
torch.save(train_dataset, '{}/train_data.pt'.format(working_folder))

# Make sure you restart the runtime, then rerun from the second cell or you will get an error

10000it [00:00, 26181.66it/s]
100%|██████████| 10/10 [00:10<00:00,  1.08s/it]


In [0]:
tokenizer.save_pretrained(working_folder)

('/content/spiece.model',
 '/content/special_tokens_map.json',
 '/content/added_tokens.json')

In [0]:
number_of_rows = len(df)
number_of_rows

10000

In [0]:
@dataclass
class T2TDataCollator(DataCollator):
    def collate_batch(self, batch: List) -> Dict[str, torch.Tensor]:
        input_ids = torch.stack([example['input_ids'] for example in batch])
        lm_labels = torch.stack([example['target_ids'] for example in batch])
        lm_labels[lm_labels[:, :] == 0] = -100
        attention_mask = torch.stack([example['attention_mask'] for example in batch])
        decoder_attention_mask = torch.stack([example['target_attention_mask'] for example in batch])
        

        return {
            'input_ids': input_ids, 
            'attention_mask': attention_mask,
            'lm_labels': lm_labels, 
            'decoder_attention_mask': decoder_attention_mask
        }

In [0]:
def load_model(first_time= os.environ['First_Time_Training_The_Model'] , working_folder=working_folder, pretrained_model_name=pretrained_model_name):
  if first_time == 'True':
    config = T5Config.from_pretrained(pretrained_model_name)
    model = T5ForConditionalGeneration.from_pretrained(pretrained_model_name, config=config)
    print('Creating a brand new T5 model')

  else:
    model = T5ForConditionalGeneration.from_pretrained(working_folder)
    print('Loading the T5 model that has been trained previously')
  train_dataset = torch.load('{}/train_data.pt'.format(working_folder))
  data_collator = T2TDataCollator()

  return model, train_dataset, data_collator

In [0]:
tokenizer = T5Tokenizer.from_pretrained(working_folder)

In [0]:
# Change the os.environment to True if you mess up with loading the model
#os.environ['First_Time_Training_The_Model'] = 'True'  ############ Only run if needs to

In [0]:
epochs = 1 ### Specifiy how many times you want the machine to loop over your data

model,train_dataset, data_collator = load_model()   #### Change to false after you run it the first time
progress = widgets.FloatProgress(value=0.1, min=0.0, max=1.0, bar_style = 'info')
training_args = TrainingArguments(
                    output_dir= working_folder,
                    overwrite_output_dir=True,
                    do_train=True,
                    num_train_epochs=epochs,   
                    per_device_train_batch_size=2, ### Specify how many chunks the data should be splitted
                    logging_steps=1000,    ### Specify how often you want to see the loss outputs
                    save_steps=-1)
trainer = Trainer(
                    model=model,
                    args=training_args,
                    data_collator=data_collator,
                    train_dataset=train_dataset,
                    prediction_loss_only=True)
  

progress.value = 0.4
p_start, p_end = 0.4, 1.
  
def progressify(f):
  def inner(*args, **kwargs):
    if trainer.epoch is not None:
      progress.value = p_start + trainer.epoch / epochs * (p_end - p_start)
      return f(*args, **kwargs)
  return inner
trainer._training_step = progressify(trainer._training_step)
trainer.train()
trainer.save_model(working_folder)
os.environ['First_Time_Training_The_Model'] = 'False'

In [0]:
# In case you loose patient, you can still save the model to keeping later
trainer.save_model(working_folder)
os.environ['First_Time_Training_The_Model'] = 'False'

In [0]:
valid_dataset = torch.load('{}/train_data.pt'.format(working_folder))    #Dont have valid data_set...
dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=32)

In [0]:
evaluate_model = T5ForConditionalGeneration.from_pretrained(working_folder).to('cuda')  # Make sure you set your runtime to GPU

In [0]:
answers = []
for batch in tqdm(dataloader):
  outs = evaluate_model.generate(input_ids=batch['input_ids'].to('cuda'), 
                        attention_mask=batch['attention_mask'].to('cuda'),
                        max_length=1500,
                        early_stopping=True)
  outs = [tokenizer.decode(ids) for ids in outs]
  answers.extend(outs)

  # You can interupt the execution. Data loaded to 'answers' array wont get lost (but make sure you have at least 5 data)

In [0]:
predictions = []
references = []
input_texts = []
for ref, pred in zip(valid_dataset, answers):
  pred = pred[4:-1]
  predictions.append(pred)
  
  input_ = tokenizer.decode(ref['input_ids'])
  input_ = ''.join(input_)[0:-3]
  input_ = re.sub('[!@#$*-]', '', input_)
  input_ = input_.lstrip().title()

  start_index = input_.index('>:')
  prefix = input_[:start_index]
  input_ = input_[start_index + 6:]

  input_texts.append(input_)

  output_ = tokenizer.decode(ref['target_ids'])
  output_ = ''.join(output_)[4:-3]
  references.append(output_)

In [0]:
for _ in range(min(5, len(answers))):
  i = random.randint(0, len(predictions))
  print('Input:             {}\nPredicted Answer:  {}\nReal Answer:       {}\n'.format(input_texts[i],predictions[i], references[i]))

Input:             How Often Is Catholic Mass Held At Notre Dame In A Wee
Predicted Answer:  Catholic Mass
Real Answer:       over 100 times

Input:             There Are How Many Dorms For Females At Notre Dam
Predicted Answer:  there are how many dorms for females at Notre Dame
Real Answer:       14

Input:             On What Date Was The Rebuilding Of The Main Building Begun At Notre Dame After The Fire That Claimed The Previou
Predicted Answer:  the library collection
Real Answer:       17th of May

Input:             In What Building Is The Current School Of Architecture Housed At Notre Dam
Predicted Answer:  School of Architecture
Real Answer:       Bond Hall

Input:             What Is The Annual Budget Of Notre Dame'S Lafortune Cente
Predicted Answer:  LaFortune
Real Answer:       $1.2 million



In [0]:
# If the question is in completed, it is because I dont have enough memmory to train the full context (maximum input length is only 1700 words). Refer to cell 4
# I only trained the model for 44 seconds, so this is why the answer isnt accurate...
# Excuse my english or grammar, I got an idea and my idea worth a million words